In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data/coup.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413600 entries, 0 to 413599
Data columns (total 37 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            413600 non-null  int64  
 1   tweet_url                     413600 non-null  object 
 2   created_at                    413600 non-null  object 
 3   parsed_created_at             413600 non-null  object 
 4   user_screen_name              413600 non-null  object 
 5   text                          413600 non-null  object 
 6   tweet_type                    413600 non-null  object 
 7   coordinates                   7 non-null       object 
 8   hashtags                      13057 non-null   object 
 9   media                         11642 non-null   object 
 10  urls                          52890 non-null   object 
 11  favorite_count                413600 non-null  int64  
 12  in_reply_to_screen_name       31057 non-null

In [5]:
# count number of different languages in data set
count_lang = df['lang'].unique()
print(len(count_lang), count_lang)

44 ['en' 'und' 'fr' 'de' 'nl' 'in' 'th' 'ja' 'pt' 'es' 'et' 'ca' 'fa' 'it'
 'tr' 'pl' 'ht' 'cy' 'ko' 'ro' 'tl' 'eu' 'zh' 'ar' 'no' 'ru' 'fi' 'pa'
 'el' 'vi' 'sv' 'iw' 'sr' 'ur' 'hu' 'lt' 'cs' 'lv' 'da' 'hi' 'is' 'sl'
 'ta' 'dv']


In [6]:
# tweets are in 44 different languages. 
# I'll be working only with tweets in English
# so, I'll drop tweets in all other languages
df = df[df.lang == 'en']
df.shape

(396586, 37)

In [ ]:
# drop unnecessary columns 
df = df.drop(['tweet_url', 'created_at', 'media', 'urls','in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_or_quote_id',
       'retweet_or_quote_screen_name', 'retweet_or_quote_user_id', 'source',
       'user_created_at', 'user_name', 'user_verified', 'user_friends_count', 'user_listed_count',
       'user_statuses_count', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'coordinates', 'lang', 'user_loc

In [7]:
# tokenize, remove stopwords, remove urls, lowercase, remove punctuation, remove numbers

import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag

stop = stopwords.words('english')

punc = list(set(string.punctuation))

def tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r'', text)

def process_text(text):
    text = remove_url(text)
    text = tokenizer(text)
    text = [word.lower() for word in text]
    text = [re.sub('[0-9]+', '', word) for word in text]
    text = [word for word in text if word not in punc]
    text = [word for word in text if word not in stop]
    text = [each for each in text if len(each) > 1]
    text = [word for word in text if ' ' not in word]
     
    return text

In [8]:
df['processed_text'] = df['text'].apply(process_text)

In [9]:
pd.set_option('display.max_colwidth', -1)
df['processed_text'][:20]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


0     [assault, democracy, sen, josh, hawley, blood, hands, capitol, coup, attempt]                                                                                                                                              
1     [call, old-fashioned, armed, insurgents, breach, capitol, building, request, president, i'd, call, attempted, coup]                                                                                                        
2     [@vritrite, @dreddersart, oh, yeah, like, politics, bad, im, like, ignore, coup, attempt, happening]                                                                                                                       
3     [ridiculously, divisive, statement, btw, gun, violence, continues, baltimore, oh, still, picked, trash]                                                                                                                    
4     [assault, democracy, sen, josh, hawley, blood, hands, capitol, coup, attempt]             

In [10]:
ready_for_pos = df['processed_text']

In [11]:
# part-of-speech tagging 
def pos_tagging(text):
    pos_tag = [pos_tag(word) for word in ready_for_pos]

df['pos_tagged'] = df.processed_text.apply(lambda x: pos_tag(x))

In [12]:
pos_tagged = df['pos_tagged']

In [13]:
# lemmatizing

from nltk.stem import WordNetLemmatizer 
wordnet = WordNetLemmatizer() 

lemmatized = [[wordnet.lemmatize(word[0]) for word in words] for words in pos_tagged]

In [14]:
df['lemmatized'] = lemmatized

In [15]:
df['final_docs'] = df['lemmatized'].apply(lambda x: " ".join(x))

In [16]:
df['final_docs'][30000:30020]

31488    call old-fashioned armed insurgent breach capitol building request president i'd call attempted coup                                                                                                
31489    anyone elected house senate challenging result presidential election congress part attempted coup inspired president resign insurrection ...                                                        
31490    twitter ball facebook none                                                                                                                                                                          
31491    almost permanent ban inciting fucking coup attempt ridiculous even ballpark jack twitter fucking disgrace complicit point blank #trumpcoupattempt                                                   
31492    @jtylerconway president u staged coup disrupt certification election cannot believe happening one always remember                                                      

In [17]:
final_docs = df['final_docs']

In [18]:
#create document term matrix with TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
# initial tuning of parameters
#set max_features to 2000 (specifies the number of most frequently occurring words for which we want to create feature vectors)
# set min_df to 5 (word must occur in at least 5 documents)
# set max_df to 0.85 (word must not occur in more than 85 percent of the documents) 

tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.85, ngram_range=(1, 2), stop_words='english')  
doc_term_matrix_1 = tfidfconverter.fit_transform(df['final_docs'].values.astype('U'))

In [19]:
doc_term_matrix_1.shape

(396586, 2000)

In [20]:
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=10)
nmf_Z = nmf_model.fit_transform(doc_term_matrix_1)

In [21]:
from sklearn.decomposition import TruncatedSVD

lsi_model = TruncatedSVD(n_components=10)
lsi_Z = lsi_model.fit_transform(doc_term_matrix_1)

In [22]:
from sklearn.decomposition import LatentDirichletAllocation
#  LDA model 
lda_model = LatentDirichletAllocation(n_components = 10, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(doc_term_matrix_1)

In [23]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, tfidfconverter )
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, tfidfconverter )
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, tfidfconverter )
print("=" * 20)

LDA Model:
Topic 0:
[('attempt', 2928.7480570645876), ('coup attempt', 2524.5785410589024), ('paul', 2216.4945536109863), ('stormed', 1750.0832652945248), ('followgoosehead', 1669.661255761883), ('stormed capitol', 1625.7455069543246), ('sorry', 1552.0071837412315), ('took', 1500.0142402168442), ('work', 1432.0448089533866), ('inciting', 1413.9455354934396)]
Topic 1:
[('white', 6671.515072843163), ('attempted', 5528.417450746919), ('ago', 5336.362101667251), ('heart', 5164.746846557939), ('black', 5145.163059865999), ('attempted coup', 5087.417675848753), ('cleaning white', 5064.0273508805285), ('worker cleaning', 5063.143437365644), ('capitol worker', 5058.807506126885), ('black capitol', 5058.263020948851)]
Topic 2:
[('couple', 5083.927488426614), ('impeachment', 4701.287679494274), ('remove', 4682.085470533492), ('impeach', 4513.209693305443), ('point', 4454.617308350619), ('staging', 4291.137321346573), ('impeach remove', 4198.89265597706), ('staging coup', 4181.27710879082), ('rem

In [24]:
import pyLDAvis.sklearn

In [25]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1      32.420952  8.115230   1       1        17.035517
4      0.648955   71.707100  2       1        12.300604
9      71.019684  9.971006   3       1        10.369589
6      3.785981   33.185135  4       1        9.568212 
2     -42.666500 -9.222070   5       1        9.453150 
3     -3.621689  -43.526516  6       1        8.987981 
8     -3.632402  -4.171708   7       1        8.501564 
0     -36.584400  35.804413  8       1        8.260808 
7      44.391380  50.615383  9       1        7.905165 
5      40.235809 -31.591665  10      1        7.617409 , topic_info=          Term         Freq        Total Category  logprob  loglift
1790  thug      5531.000000  5531.000000  Default  30.0000  30.0000
1975  worker    5447.000000  5447.000000  Default  29.0000  29.0000
470   couple    5510.000000  5510.000000  Default  28.0000  28.0000
314   cleaning  5489.000000  5489.000000  Default  27.0000  27.0000
1436  remove    5075.000000  5075.000000  Default  26.0000  26.0000
...      ...            ...          ...      ...      ...      ...
184   biden     1541.099707  1844.761166  Topic10 -4.2621   2.3949 
985   know      1933.523733  2828.590182  Topic10 -4.0353   2.1943 
488   cruz      1404.497365  2022.995380  Topic10 -4.3549   2.2098 
1222  order     1261.386667  1707.700197  Topic10 -4.4624   2.2718 
820   hawley    1342.099853  4784.660020  Topic10 -4.4004   1.3036 

[438 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
11    9      0.999316  act             
12    9      0.998426  act insurrection
23    6      0.999330  actual          
24    6      0.999158  actual coup     
26    4      0.999022  administration  
...  ..           ...          ...     
1974  2      0.999351  work trump      
1975  9      0.999842  worker          
1976  1      0.999745  worker cleaning 
1977  9      0.999607  working         
1978  9      0.999300  working coup    

[601 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 10, 7, 3, 4, 9, 1, 8, 6])